# Cluster Buster Pipeline

## Gather and Clean Data

In [11]:
# testing create_data.py
wrap_text="bash create_data.sh data/ndd_genes_pos_uniq.txt /data/CARD_AA/projects/2023_05_JM_gt_clusters/snp_metrics/snp_metrics_204842400049/Sample_ID=204842400049_R01C01 /data/CARD_AA/projects/2023_05_JM_gt_clusters/snp_metrics/gp2_sampleids_release6_with_ancestries.tab /data/GP2/raw_genotypes/snp_metrics /data/CARD_AA/projects/2023_05_JM_gt_clusters/capstone/data/"
!sbatch --cpus-per-task=2 --mem=20g --time=2:00:00 --wrap="$wrap_text"

24787285


## Create, Train, and Save Neural Network

In [ ]:
# runs create_model.py in focalloss environment 
! sbatch --mem=40g --cpus-per-task=4 --partition=gpu --gres=gpu:v100x:2,lscratch:200 --time=1-0 --mail-type=BEGIN,END create_model.sh

## Render Predictions

### on all no-NC data

In [ ]:
! cat data/train.csv data/val.csv data/test.csv > no_nc.csv

In [26]:
wrap_text="bash src/predictions.sh model/gt_model.keras data/no_nc.csv data/no_nc_predictions.csv"
!sbatch --cpus-per-task=2 --mem=20g --time=2:00:00 --wrap="$wrap_text"

24805256


### on test data

In [28]:
wrap_text="bash src/predictions.sh model/gt_model.keras data/test.csv data/test_predictions.csv"
!sbatch --cpus-per-task=2 --mem=20g --time=2:00:00 --wrap="$wrap_text"

24805257


### on no-calls

In [29]:
wrap_text="bash src/predictions.sh model/gt_model.keras data/nc_test.csv data/nc_predictions.csv"
!sbatch --partition=gpu --cpus-per-task=2 --mem=20g --gres=gpu:v100x:2,lscratch:200 --time=2:00:00 --wrap="$wrap_text"

24805261


## Mark Outliers

In [8]:
wrap_text="bash src/outlier_detection.sh data/no_nc_predictions.csv data/nc_predictions.csv 0.5 2 data/GP2_release6_maf.frq /data/CARD_AA/projects/2023_05_JM_gt_clusters/snp_metrics/hg38_maf/ data/new_outliers.csv"
!sbatch --cpus-per-task=2 --mem=40g --time=4:00:00 --wrap="$wrap_text"

25618447


In [7]:
! scancel 25618445

## Visualize SNPs with App

In [ ]:
! streamlit run visualizing_snps_app.py --server.enableXsrfProtection false




  You can now view your Streamlit app in your browser.

  Network URL: http://127.0.0.1:8501
  External URL: http://128.231.2.73:8501

